In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, f1_score, accuracy_score

# Load data
train = pd.read_csv("/kaggle/input/dataset/train.csv")
test = pd.read_csv("/kaggle/input/dataset/test.csv")
sample_submission = pd.read_csv("/kaggle/input/dataset/sample_submission.csv")

In [3]:
# Drop unused columns
train = train.drop(columns=["Date"])
test_ids = test["ID"]
test = test.drop(columns=["Date", "ID"])

# Features & target
X = train.drop("target", axis=1)
y = train["target"]

# Scaling
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
test_scaled = scaler.transform(test)

# Train/validation split
X_train, X_val, y_train, y_val = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

In [4]:
log_model = LogisticRegression(max_iter=1000)
log_model.fit(X_train, y_train)
log_preds = log_model.predict(X_val)

print("Logistic Regression Report:")
print(classification_report(y_val, log_preds))

Logistic Regression Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    325118
           1       0.55      0.07      0.12      2767

    accuracy                           0.99    327885
   macro avg       0.77      0.53      0.56    327885
weighted avg       0.99      0.99      0.99    327885



In [5]:
from sklearn.ensemble import RandomForestClassifier

rf_model = RandomForestClassifier(n_estimators=50, n_jobs=-1, random_state=42)
rf_model.fit(X_train, y_train)
rf_preds = rf_model.predict(X_val)

from sklearn.metrics import classification_report
print("Random Forest Report:")
print(classification_report(y_val, rf_preds))

Random Forest Report:
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    325118
           1       0.50      0.25      0.33      2767

    accuracy                           0.99    327885
   macro avg       0.75      0.62      0.66    327885
weighted avg       0.99      0.99      0.99    327885



In [6]:
# Evaluate both models
log_f1 = f1_score(y_val, log_preds)
rf_f1 = f1_score(y_val, rf_preds)

print(f"Logistic Regression F1 Score: {log_f1:.4f}")
print(f"Random Forest F1 Score: {rf_f1:.4f}")

# Choose the better model
if rf_f1 > log_f1:
    print("Using Random Forest for final prediction")
    final_preds = rf_model.predict(test_scaled)
else:
    print("Using Logistic Regression for final prediction")
    final_preds = log_model.predict(test_scaled)

Logistic Regression F1 Score: 0.1214
Random Forest F1 Score: 0.3342
Using Random Forest for final prediction


In [8]:
# Create new submission DataFrame with correct shape
submission = pd.DataFrame({
    "ID": test_ids,
    "target": final_preds
})
submission.to_csv("submission.csv", index=False)